In [ ]:
import pickle as pkl
import pandas as pd

In [ ]:
file_path = "results/adresso/mmse/scores_03101732.pkl"
with open(file_path, "rb") as file:
    results = pkl.load(file=file)
    desc = results[0]
    scores_list = results[1:]
print(desc)

for changing desc if needed

In [ ]:
# change_desc = False
# if change_desc:
#     new_desc = f'200, 50 k=11. removed final bn. fixed dense output'
#     results[0] = new_desc
#     with open(file_path, "wb") as file:
#         pkl.dump(obj=results, file=file)

In [ ]:
for i, scores in enumerate(scores_list):
    print(abs(scores['nona euclidean mlp'][0]))

print(i+1, 'splits')

In [ ]:
scores = {method.split(" mlp")[0]: [abs(scores[method][0]) for scores in scores_list] for method in scores_list[0] if "mlp" in method and "cos" not in method}

In [ ]:
bplot = pd.DataFrame(scores).boxplot(rot=30)
# bplot.set_ylabel('MSE')

In [ ]:
pd.DataFrame(scores).median()

In [ ]:
times = {method: [scores[method][1] for scores in scores_list] for method in scores_list[0]}

In [ ]:
pd.DataFrame(times).median() / 3600